In [1]:
# import findspark
# # 可在环境变量中进行设置，即PATH中加入如下地址
# findspark.init("F:/spark-2.4.3-bin-hadoop2.7")
# from pyspark.sql import SparkSession
from pyspark import SparkContext
# from pyspark import SparkConf

In [2]:
# 创建sc
sc = SparkContext('local','Simple')

In [3]:
# 从文件系统中加载数据创建RDD
fileRDD = sc.textFile('../../../dataset/word.txt')

In [4]:
# 通过并行集合（数组）常见RDD,可以调用SparkContext的parallelize方法，在Driver中一个已经存在的集合(数组)上创建
nums = [1,2,3,4,5]
rdd = sc.parallelize(nums)

#### 转换操作
每一次转换操作都会产生不同的RDD，供给下一个转换使用，整个转换过程只是记录了转换的轨迹，并不会发生真正的计算，只有遇到行到操作时，才会发生真正的计算。<br>
常见的转换操作（Transformation API）：
* filter(func)：筛选出满足函数func的元素，并返回一个新的数据集
* map(func)：将每个元素传递到函数func中，并将结果返回为一个新的数据集
* flatMap(func)：与map()相似，但每个输入元素都可以映射到0或多个输出结果
* groupByKey()：应用于(K,V)键值对的数据集时，返回一个新的(K, Iterable)形式的数据集
* reduceByKey(func)：应用于(K,V)键值对的数据集时，返回一个新的(K, V)形式的数据集，其中的每个值是将每个key传递到函数func中进行聚合

#### 行动操作
行动操作是真正触发计算的地方。Spark程序执行到行动操作时，才会执行真正的计算，从文件中加载数据，完成一次又一次转换操作，最终，完成行动操作得到结果。<br>
下面列出一些常见的行动操作（Action API）：
* count() 返回数据集中的元素个数
* collect() 以数组的形式返回数据集中的所有元素
* first() 返回数据集中的第一个元素
* take(n) 以数组的形式返回数据集中的前n个元素
* reduce(func) 通过函数func（输入两个参数并返回一个值）聚合数据集中的元素
* foreach(func) 将数据集中的每个元素传递到函数func中运行*

#### 惰性机制

In [5]:
lines = sc.textFile('../../../dataset/word.txt')
lineLengths = lines.map(lambda s : len(s))
totalLength = lineLengths.reduce(lambda a ,b : a + b)

#### 持久化

In [10]:
list_tmp = ["Hadoop","Spark","Hive"]
rdd = sc.parallelize(list_tmp)
print(rdd.count()) #行动操作，触发一次真正从头到尾的计算

3


In [13]:
print(','.join(rdd.collect())) #行动操作，触发一次真正从头到尾的计算

Hadoop,Spark,Hive


In [14]:
rdd.cache() #会调用persist(MEMORY_ONLY),但是，语句执行到这里，并不会缓存rdd,这是rdd还没有被计算生成

ParallelCollectionRDD[6] at parallelize at PythonRDD.scala:195

In [15]:
print(rdd.count()) # 第一次行到操作，触发一次真正从头到尾的计算，这时才会执行上面的rdd.cache(),吧这个rdd放到缓存中

3


In [17]:
print(','.join(rdd.collect())) #行动操作，不需要触发从头到尾的计算，只需要重复使用上面缓存中的rdd
# 最后使用unpersist()方法手动地把持久化的RDD从缓存中移除。

Hadoop,Spark,Hive


#### 分区 

In [18]:
array = [1,2,3,4,5]
rdd = sc.parallelize(array,2)